# Generate Dataloader From Files

In [1]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from pycocotools.coco import COCO
import os
import numpy as np
import json

with open('./annotations/person_keypoints_train2017.json', 'r') as f:
    d = json.load(f)
coco = COCO('./annotations/instances_train2017.json')

category_ids = coco.getCatIds()
NUM_CLASSES = len(category_ids)
MAX_OBJECTS = 62
BATCH_SIZE = 2

imgIds = coco.getImgIds()
filenames = os.listdir('./images/train2017/')
bounding_boxes = []
object_classes = []
indexes = []
for id in imgIds:
    img_dict = coco.loadImgs(id)[0]
    annotations = coco.loadAnns(coco.getAnnIds([id], iscrowd=False))
    if len(annotations) > 0: 
        indexes.append(filenames.index(img_dict['file_name']))
        boxes = []
        for annotation in annotations:
            box = annotation['bbox']
            boxes.append([box[0]/img_dict['width'], box[1]/img_dict['height'], (box[0]+box[2])/img_dict['width'], (box[1]+box[3])/img_dict['height']])
        boxes_length = len(boxes)
        for _ in range(MAX_OBJECTS - boxes_length):
            boxes.append([0.0, 0.0, 0.0, 0.0])
        bounding_boxes.append(boxes)
        classes = []
        for annotation in annotations:
            classes.append(category_ids.index(annotation['category_id']))
        classes_length = len(classes)
        for _ in range(MAX_OBJECTS - classes_length):
            classes.append(NUM_CLASSES)
        object_classes.append(classes)

images = ImageFolder('./images/', transform=transforms.Compose([
        transforms.Resize((320,320)), transforms.ToTensor()
        ]))
images_loader = DataLoader(Subset(images, indexes), batch_size=BATCH_SIZE, shuffle=False)
boxes_loader = DataLoader(np.array(bounding_boxes), batch_size=BATCH_SIZE, shuffle=False)
classes_loader = DataLoader(np.array(object_classes), batch_size=BATCH_SIZE, shuffle=False)

# Train DETR

In [ ]:
import torch 
from torch.optim import Adam
from detr import DETR, DETRLoss

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DETR(num_encoder_layers=6, num_decoder_layers=6, num_classes=NUM_CLASSES, max_objects=MAX_OBJECTS).to(DEVICE)
criterion = DETRLoss(0.5, 0.5, NUM_CLASSES)
optimizer = Adam(model.parameters(), lr=0.0005)

EPOCHS = 20
for epoch in range(EPOCHS):
    loss_sum = 0
    i = 0
    for ((images,_), bboxes, classes) in zip(images_loader, boxes_loader, classes_loader):
        images, bboxes, classes = images.to(DEVICE), bboxes.to(DEVICE), classes.to(DEVICE)
        bb_predicted, c_predicted = model(images)
        loss = criterion(bboxes, bb_predicted, classes, c_predicted)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        i += 1
    print(f'epoch {epoch}   loss {loss_sum / i}')